<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/opengov_country.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install ckanapi

In [55]:
from ckanapi import RemoteCKAN
import pandas as pd
import os
import zipfile
import yaml
import re
import unicodedata

In [56]:
# Access resource data via a web API
rc = RemoteCKAN('https://open.canada.ca/data/en/')
APIID = "bdb33e8c-53ef-4bae-9493-35f343191c02"

# Initialize an empty list to store all records
all_records = []

# Set the initial offset and desired limit
offset = 0
limit = 100  # Or any desired number of records per request

while True:
    result = rc.action.datastore_search(
        resource_id=APIID,
        limit=limit,
        offset=offset,
    )

    # Add the retrieved records to the list
    all_records.extend(result['records'])

    # If the number of records returned is less than the limit,
    # it means we've reached the end
    if len(result['records']) < limit:
        break

    # Increment the offset for the next request
    offset += limit

# Create the DataFrame from all the retrieved records
df = pd.DataFrame(all_records)

# Add a new 'title' column using the content of 'GC_NM_AB_EN'
df['title'] = df['GC_NM_AB_EN']

# Remove French accent characters from the 'title' column
df['title'] = df['title'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('ascii'))

print(df.count())
print(df.head())

_id                380
GC_HIST_ID         380
GC_ID              380
GC_NM_OFF_EN       380
GC_NM_OFF_FR       380
GC_NM_AB_EN        380
GC_NM_AB_FR        380
STAT_CD            380
STAT_DESC_EN       380
STAT_DESC_FR       380
TYPE_CD            380
TYPE_DESC_EN       380
TYPE_DESC_FR       380
RECENT_IND         380
EFF_DT             380
EFF_FN_DT          380
OBSERVATIONS_EN    380
OBSERVATIONS_FR    380
GC_PAR_ID            6
GC_JUR_ID           53
ISO_ALPHA_2_CD     367
ISO_ALPHA_3_CD     368
UN_ONU_CD          363
MODIF_DT           380
title              380
dtype: int64
   _id GC_HIST_ID  GC_ID                                  GC_NM_OFF_EN  \
0    1    1000110  10001                          the Republic of Fiji   
1    2    1000210  10002  South Georgia and the South Sandwich Islands   
2    3    1000310  10003                the People’s Republic of China   
3    4    1000410  10004                      the Republic of Slovenia   
4    5    1000508  10005                  

In [65]:
output = df[df['STAT_CD'] == "1"]
clean_output = output[['GC_NM_OFF_EN', 'GC_NM_OFF_FR', 'GC_ID', 'UN_ONU_CD', 'ISO_ALPHA_2_CD','GC_NM_AB_FR', 'GC_NM_AB_EN','MODIF_DT','EFF_DT']].rename(
    columns={
        'GC_NM_OFF_EN': 'title',
        'GC_ID': 'id',
        'UN_ONU_CD': 'cd_un',
        'ISO_ALPHA_2_CD': 'cd_iso2',
        'GC_NM_OFF_FR': 'french',
        'GC_NM_AB_EN':'aliases',
         'GC_NM_AB_FR':'aliases_fr',
        'MODIF_DT':'modified',
        'EFF_DT':'effectiveDate'
    }
)

# Create a new column 'identifier' with the specified starting value
alias_start = 2025010312471
clean_output['identifier'] = range(alias_start, alias_start + len(clean_output))


# Convert to datetime if necessary
clean_output['modified'] = pd.to_datetime(clean_output['modified'])
clean_output['effectiveDate'] = pd.to_datetime(clean_output['effectiveDate'])

# Format the columns
clean_output['modified'] = clean_output['modified'].dt.strftime('%Y-%m-%d')
clean_output['effectiveDate'] = clean_output['effectiveDate'].dt.strftime('%Y-%m-%d')
clean_output["identifier"] = clean_output["identifier"].apply(lambda x: f"\"{x}\"" if pd.notna(x) else x)
clean_output["aliases"] = clean_output["aliases"].apply(lambda x: f"\"{x}\"" if pd.notna(x) else x)
clean_output["aliases_fr"] = clean_output["aliases_fr"].apply(lambda x: f"\"{x}\"" if pd.notna(x) else x)
clean_output["french"] = clean_output["french"].apply(lambda x: f"\"{x}\"" if pd.notna(x) else x)

# Combine 'aliases', 'aliases_fr', and 'french' into a new column 'aliases_combined'
clean_output['aliases_combined'] =  clean_output['identifier'].astype(str) + ', '+clean_output['aliases'].astype(str) + ', ' + clean_output['aliases_fr'].astype(str) + ', ' + clean_output['french'].astype(str)
clean_output = clean_output.drop(columns=['aliases', 'aliases_fr', 'identifier'])
clean_output = clean_output.rename(columns={'aliases_combined': 'aliases'})

clean_output["aliases"] = clean_output["aliases"].apply(lambda x: f"[{x}]" if pd.notna(x) else x)


print(clean_output.head())
print(clean_output.count())

                                          title  \
0                          the Republic of Fiji   
1  South Georgia and the South Sandwich Islands   
2                the People’s Republic of China   
3                      the Republic of Slovenia   
6                      the Republic of Kiribati   

                                         french     id cd_un cd_iso2  \
0                     "la République des Fidji"  10001   242      FJ   
1  "Géorgie du Sud-et-les Îles Sandwich du Sud"  10002   239      GS   
2            "la République populaire de Chine"  10003   156      CN   
3                   "la République de Slovénie"  10004   705      SI   
6                   "la République de Kiribati"  10005   296      KI   

     modified effectiveDate                                            aliases  
0  2023-11-28    1970-01-01  ["2025010312471", "Fiji", "Fidji", "la Républi...  
1  2023-11-28    1993-01-01  ["2025010312472", "South Georgia and the South...  
2  2023-11-28    

In [67]:
import os

# Create a new directory for Markdown files with a unique name
dir_name = "markdown_files"
i = 1
while os.path.exists(dir_name):
    dir_name = f"markdown_files_{i}"
    i += 1
os.makedirs(dir_name)

# Function to create markdown content with bullet points
def create_markdown_content(row):
    markdown_content = f"---\ntitle: {row['title']}\ntags: [place/country, gac]\ntype: \"[[Country]]\"\n"  # Changed 'type' to 'title'
    for column in clean_output.columns:  # Changed grouped_df to clean_output
        if column not in ['title']:  # Changed columns to exclude
            value = row[column]
            # Check if value is a Series and handle it appropriately
            if isinstance(value, pd.Series):
                # Use any() to check if any value in the Series is not NA and not empty string
                if value.notna().any() and value.astype(str).str.strip().ne('').any():
                    markdown_content += f"{column}: {','.join(value.astype(str).tolist())}\n"
            # If not a Series, use pd.notna as before
            elif pd.notna(value) and value != '':
                markdown_content += f"{column}: {value}\n"
    markdown_content += "---"
    return markdown_content

def create_markdown_file(row):
    file_name = os.path.join(dir_name, f"{row['title']}.md")  # Changed 'type' to 'title'
    with open(file_name, "w") as f:
        markdown_content = create_markdown_content(row)
        f.write(markdown_content)
    return alias_start + 1

# Apply with alias_start and accumulate the updated alias_start
alias_start = clean_output.apply(lambda row: create_markdown_file(row), axis=1).iloc[-1]

print(f"Markdown files created successfully in directory: {dir_name}")

Markdown files created successfully in directory: markdown_files_7


In [68]:
import zipfile
# Zip the output folder
zip_file_name = f"{dir_name}.zip"
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for root, _, files in os.walk(dir_name):  # change ufolder_name to dir_name
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), dir_name))  # change ufolder_name to dir_name

print(f"Output files zipped to: {zip_file_name}")

Output files zipped to: markdown_files_7.zip
